<a href="https://colab.research.google.com/github/appliedcode/mthree-c422/blob/mthree-c422-Likhitha/prompt_practice_md.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
import torch

# Select device (GPU if available)
device = 0 if torch.cuda.is_available() else -1

# Pipelines
zero_shot_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
generator = pipeline("text-generation", model="distilgpt2", device=device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [2]:
print("\n=== Zero-Shot: News Headline Classification ===")
texts = [
    "NASA launches new mission to search for exoplanets.",
    "The city council approved new recycling rules yesterday.",
    "Barcelona defeats Real Madrid in a thrilling match."
]
candidate_labels = ["science", "sports", "politics", "environment"]

for text in texts:
    result = zero_shot_classifier(text, candidate_labels)
    print(f"Headline: {text}")
    print(f"Prediction: {result['labels'][0]} (score={result['scores'][0]:.2f})\n")

print("\n=== Zero-Shot: Sentiment Classification ===")
texts = [
    "What a disappointing experience, nothing went right.",
    "Absolutely loved the atmosphere and the food was perfect!",
    "It was okay, nothing stood out as good or bad."
]
candidate_labels = ["positive", "negative", "neutral"]

for text in texts:
    result = zero_shot_classifier(text, candidate_labels)
    print(f"Text: {text}")
    print(f"Prediction: {result['labels'][0]} (score={result['scores'][0]:.2f})\n")



=== Zero-Shot: News Headline Classification ===
Headline: NASA launches new mission to search for exoplanets.
Prediction: science (score=0.91)

Headline: The city council approved new recycling rules yesterday.
Prediction: environment (score=0.88)

Headline: Barcelona defeats Real Madrid in a thrilling match.
Prediction: sports (score=0.93)


=== Zero-Shot: Sentiment Classification ===
Text: What a disappointing experience, nothing went right.
Prediction: negative (score=1.00)

Text: Absolutely loved the atmosphere and the food was perfect!
Prediction: positive (score=0.99)

Text: It was okay, nothing stood out as good or bad.
Prediction: neutral (score=0.95)



In [3]:
print("\n=== Few-Shot: Intent Classification ===")
prompt = """Classify each customer support inquiry as Order, Technical Issue, or Complaint.

Inquiry: "My package hasn't arrived and it's a week late."
Intent: Complaint

Inquiry: "Can I change my delivery address after ordering?"
Intent: Order

Inquiry: "The app crashes when I try to log in."
Intent: Technical Issue

Inquiry: "The product is broken, please help."
Intent:"""
resp = generator(prompt, max_new_tokens=10, temperature=0.2)
print(resp[0]['generated_text'])

print("\n=== Few-Shot: Movie Genre Classification ===")
prompt = """Classify the following movie plots as Comedy, Drama, or Action.

Plot: "A group of friends go on a hilarious road trip and get into wild situations."
Genre: Comedy

Plot: "A detective teams up with a spy to stop an international heist."
Genre: Action

Plot: "A family comes together during a difficult time to heal old wounds."
Genre: Drama

Plot: "Two rival chefs compete to win the city's food festival."
Genre:"""
resp = generator(prompt, max_new_tokens=10, temperature=0.2)
print(resp[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Few-Shot: Intent Classification ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify each customer support inquiry as Order, Technical Issue, or Complaint.

Inquiry: "My package hasn't arrived and it's a week late."
Intent: Complaint

Inquiry: "Can I change my delivery address after ordering?"
Intent: Order

Inquiry: "The app crashes when I try to log in."
Intent: Technical Issue

Inquiry: "The product is broken, please help."
Intent: Complaint
Inquiry: "My package

=== Few-Shot: Movie Genre Classification ===
Classify the following movie plots as Comedy, Drama, or Action.

Plot: "A group of friends go on a hilarious road trip and get into wild situations."
Genre: Comedy

Plot: "A detective teams up with a spy to stop an international heist."
Genre: Action

Plot: "A family comes together during a difficult time to heal old wounds."
Genre: Drama

Plot: "Two rival chefs compete to win the city's food festival."
Genre: Action
Plot: "A family of friends go


In [5]:
# ensure generator exists (create if not)
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
generator = pipeline("text-generation", model="distilgpt2", device=device)

# --- Prompt 1 (math CoT) ---
prompt = """Q: There are 18 oranges. If 6 are eaten and then 4 more are bought, how many are there now?
Let's think step by step.

1. Start with 18 oranges.
2. 6 are eaten: 18 - 6 = 12 oranges left.
3. 4 more are bought: 12 + 4 = 16 oranges.

Answer: 16

Q: John has 24 candies. He gives 7 to his friend and then buys 5 more. How many candies does John have?
Let's think step by step.
"""
# Greedy decoding (no sampling)
resp = generator(prompt, max_new_tokens=50, do_sample=False)
print(resp[0]['generated_text'])

# --- Prompt 2 (everyday reasoning) ---
prompt2 = """Q: Amy has 20 dollars. She spends 8 dollars on lunch and 5 dollars on a gift. How much money does she have left?
Let's think step by step.

1. Start with 20 dollars.
2. Spend 8 dollars: 20 - 8 = 12 dollars left.
3. Spend 5 dollars: 12 - 5 = 7 dollars left.

Answer: 7

Q: Ben has 15 chocolate bars. He shares 5 with his friends and then buys 3 more. How many does he have now?
Let's think step by step.
"""
resp2 = generator(prompt2, max_new_tokens=50, do_sample=False)
print(resp2[0]['generated_text'])


Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Q: There are 18 oranges. If 6 are eaten and then 4 more are bought, how many are there now?
Let's think step by step.

1. Start with 18 oranges.
2. 6 are eaten: 18 - 6 = 12 oranges left.
3. 4 more are bought: 12 + 4 = 16 oranges.

Answer: 16

Q: John has 24 candies. He gives 7 to his friend and then buys 5 more. How many candies does John have?
Let's think step by step.
1. Start with 18 oranges.
2. 6 are eaten: 18 - 6 = 12 oranges left.
3. 4 more are bought: 12 + 4 = 16 oranges.
Answer: 16
Q: John has 24 candies.
Q: Amy has 20 dollars. She spends 8 dollars on lunch and 5 dollars on a gift. How much money does she have left?
Let's think step by step.

1. Start with 20 dollars.
2. Spend 8 dollars: 20 - 8 = 12 dollars left.
3. Spend 5 dollars: 12 - 5 = 7 dollars left.

Answer: 7

Q: Ben has 15 chocolate bars. He shares 5 with his friends and then buys 3 more. How many does he have now?
Let's think step by step.
1. Start with 20 dollars.
2. Spend 5 dollars: 12 - 5 = 7 dollars left.
3. Spen